# Demand Feature Pipeline Notebook

This notebook processes demand data and uploads it to the Hopsworks feature store. It replicates the functionality of the feature_pipeline.py script in an interactive format.

In [1]:
# Import necessary libraries
import pandas as pd
import hopsworks
import os
from datetime import datetime
from dotenv import load_dotenv

/Users/lex/miniforge3/envs/python310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Environment Variables

We'll load environment variables for Hopsworks connection credentials.

In [ ]:
# Load environment variables
load_dotenv()

# Configure parameters (these can be modified as needed)
project_name = 'many_models'
feature_group_name = 'demand_features'
version = 1

## Connect to Hopsworks Feature Store

Establish connection to the Hopsworks Feature Store using credentials from environment variables.

In [3]:
print("Connecting to Hopsworks Feature Store")

# Connect to Hopsworks
project = hopsworks.login(
    host=os.getenv("HOST"),
    port=os.getenv("PORT"),
    api_key_value=os.getenv("HOPSWORKS_API_KEY"),
    project=project_name or os.getenv("PROJECT")
)

fs = project.get_feature_store()
print(f"Connected to feature store in project: {project_name}")

Connecting to Hopsworks Feature Store
2025-05-09 10:04:46,780 INFO: Initializing external client
2025-05-09 10:04:46,781 INFO: Base URL: https://demo.hops.works:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.1) by running 'pip install hopsworks==4.1.*'


2025-05-09 10:04:48,063 INFO: Python Engine initialized.

Logged in to project, explore it here https://demo.hops.works:443/p/14455
Connected to feature store in project: many_models


## Load Source Data

Load the demand data from CSV file and prepare it for the feature store.

In [4]:
print("Loading source data")
demand_df = pd.read_csv('../data/demand_qty_item_loc.csv')

# Display first few rows to inspect the data
display(demand_df.head())

Loading source data


,sp_id,loc_id,time_bucket,repetitive_demand_quantity
0,9684698,3,202104,55.0
1,9684698,3,202105,117.0
2,9684698,3,202106,62.0
3,9684698,3,202107,45.0
4,9684698,3,202108,77.0


In [5]:

# Add datetime column for feature store
demand_df['datetime'] = datetime.now()

# Display the transformed dataframe
display(demand_df.head())

,sp_id,loc_id,time_bucket,repetitive_demand_quantity,datetime
0,9684698,3,202104,55.0,2025-05-09 10:05:45.219201
1,9684698,3,202105,117.0,2025-05-09 10:05:45.219201
2,9684698,3,202106,62.0,2025-05-09 10:05:45.219201
3,9684698,3,202107,45.0,2025-05-09 10:05:45.219201
4,9684698,3,202108,77.0,2025-05-09 10:05:45.219201


## Create Feature Group and Upload Data

Define the feature group schema and upload the prepared data to the feature store.

In [ ]:
print("⬆ Creating/getting feature group")
# Define the feature group
demand_fg = fs.get_or_create_feature_group(
    name=feature_group_name,
    version=version,
    description="Item demand by location and time",
    primary_key=['sp_id', 'loc_id', 'time_bucket'],
    event_time='datetime',
)

⬆ Creating/getting feature group


TypeError: FeatureStore.get_or_create_feature_group() missing 1 required positional argument: 'version'

In [ ]:
print("⬆ Uploading data to the Feature Store")
# Upload data to the feature store
demand_fg.insert(demand_df, write_options={"wait_for_job": True})
print("Feature pipeline completed successfully")

## Verify Feature Group 

Retrieve and inspect the feature group schema and data to verify successful upload.

In [ ]:
# Retrieve feature descriptions
print("Feature Group Schema:")
display(demand_fg.describe())

In [ ]:
# Sample data from the feature group
print("Sample Data from Feature Group:")
sample_data = demand_fg.read()
display(sample_data.head(10))